In [19]:
__file__ = "__init__.py"

In [20]:
import sys, os, json5
from pathlib import Path
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import TFT5ForConditionalGeneration, T5Tokenizer

project_root = Path(__file__).resolve().parents[1]
sys.path.append(str(project_root))

from utils.qgene import generate_text

In [21]:
paths = {
    "processed": os.path.abspath(f"{project_root}/data/storage/processed"),
    "qfragments": os.path.abspath(f"{project_root}/intents/qfragments.json"),
    "questions": os.path.abspath(f"{project_root}/intents/questions.csv"),
    "qtrain": os.path.abspath(f"{project_root}/intents/qtrain.csv"),
    "models": os.path.abspath(f"{project_root}/models/t5-small"),
    "results": os.path.abspath(f"{project_root}/training/results"),
}

In [22]:
def load_dataset(num_samples: int=1000, batch_size: int=32, shuffle_buffer: int=1000):
    """
    Sinh dữ liệu bằng generate_text, chuyển đổi sang tf.data.Dataset.
    """
    df = generate_text(num = num_samples, choice = "s1", save = False)
    # Giả sử ta cần tạo dataset cho phần input (question) và target (chuỗi chứa các component)
    # Ví dụ: input là "Extract component: {question}" và target là chuỗi có định dạng "BRAND: ...; CPU: ...; ..."
    inputs = df["question"].apply(lambda x: "Extract component: " + x).tolist()
    targets = df.apply(
        lambda row: (
            f"BRAND: {row['brand']}; "
            f"PRICE: {row['price']}; "
            f"RAM: {row['ram']}; "
            f"GPU: {row['gpu']}; "
            f"CPU: {row['cpu']}; "
            f"DISPLAY: {row['display']}; "
            f"REFRESH_RATE: {row['refresh rate']}"
        ).strip(),
        axis=1
    ).tolist()
    
    dataset = tf.data.Dataset.from_tensor_slices((inputs, targets))
    dataset = dataset.shuffle(shuffle_buffer).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [23]:
ds = load_dataset(num_samples=1000, batch_size=16)
for batch in ds.take(1):
    inp, tgt = batch
    print("Input mẫu:", inp.numpy()[0])
    print("Target mẫu:", tgt.numpy()[0])

Input mẫu: b'Extract component: Looking for information on laptops assembled by msi, having ryzen 5 pro 8th combined with nvidia geforce rtx 3080 ti joined by 96gb as well 3k display as well as 144hz suitable for running high-performance data mining software My budget cap is 3027 USD..'
Target mẫu: b'BRAND: msi; PRICE: My budget cap is 3027 USD.; RAM: 96gb; GPU: nvidia geforce rtx 3080 ti; CPU: ryzen 5 pro 8th; DISPLAY: 3k display; REFRESH_RATE: 144hz'


In [24]:
def build_projection_layer(input_dim=256, output_dim=512):
    """
    Xây dựng một projection layer sử dụng tf.keras.layers.Dense để chuyển từ 256 chiều lên 512 chiều.
    """
    # Lưu ý: Không sử dụng activation, lớp này chỉ thực hiện phép biến đổi tuyến tính.
    projection_layer = tf.keras.layers.Dense(output_dim, input_shape=(input_dim,))
    return projection_layer

vocab_size = 10
embedding_matrix = np.random.randn(vocab_size, 256).astype("float32")
projection = build_projection_layer(256, 512)
projected = projection(embedding_matrix)
print("Kích thước sau projection:", projected.shape)  # Kỳ vọng (10, 512)

Kích thước sau projection: (10, 512)


In [25]:
def integrate_custom_embeddings(embedding_matrix, projection_layer, t5_model, union_vocab):
    """
    Cập nhật embedding layer của T5 với các vector từ embedding_matrix sau khi qua projection layer.
    """
    # Resize embedding của T5 nếu cần (đảm bảo số lượng token khớp với union_vocab)
    new_vocab_size = len(union_vocab)
    t5_model.resize_token_embeddings(new_vocab_size)
    
    # Chuyển đổi embedding matrix sang tensor và qua projection layer
    custom_embeddings = tf.convert_to_tensor(embedding_matrix, dtype=tf.float32)    # (vocab_size, 256)
    projected_embeddings = projection_layer(custom_embeddings)                      # (vocab_size, 512)
    
    # Cập nhật biến shared embeddings của T5 bằng cách dùng set_weights
    t5_model.shared.set_weights([projected_embeddings.numpy()])
    
    # Đảm bảo rằng embedding layer là trainable để fine-tune cùng với mô hình T5
    t5_model.shared.trainable = True
    return t5_model


In [26]:
def main():
    # Thiết lập tf.data.Dataset
    batch_size = 32
    dataset = load_dataset(num_samples=1000, batch_size=batch_size)
    
    # Tải tokenizer T5
    t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
    
    # Xây dựng union vocabulary
    # Ở đây, ta sẽ sử dụng vocabulary của tokenizer T5 cho demo.
    # Nếu cần kết hợp với từ điển riêng từ generate_text, bạn có thể mở rộng hàm này.
    union_vocab = t5_tokenizer.get_vocab()  # dict mapping token -> index
    print("Kích thước vocabulary:", len(union_vocab))
    
    # Huấn luyện embedding module:
    # Ở đây, để demo, ta tạo embedding matrix ngẫu nhiên với shape (vocab_size, 256)
    vocab_size = len(union_vocab)
    embedding_matrix = np.random.randn(vocab_size, 256).astype("float32")
    
    # Xây dựng projection layer (256 -> 512)
    projection_layer = build_projection_layer(input_dim=256, output_dim=512)
    
    # Tải mô hình T5 phiên bản TensorFlow
    t5_model = TFT5ForConditionalGeneration.from_pretrained("t5-small")
    
    # Tích hợp custom embeddings vào T5
    t5_model = integrate_custom_embeddings(embedding_matrix, projection_layer, t5_model, union_vocab)
    print("Đã cập nhật embedding của T5 với custom embeddings qua projection layer.")
    
    # Ví dụ fine-tuning/inference mẫu:
    # Lấy một batch từ dataset
    for batch in dataset.take(1):
        inputs, targets = batch  # inputs và targets là các chuỗi
        # Mã hóa inputs sử dụng tokenizer của T5
        input_list = [s.decode("utf-8") for s in inputs.numpy()]
        input_encodings = t5_tokenizer(input_list, return_tensors="tf", padding=True, truncation=True)

        # Sử dụng mô hình T5 để generate output
        outputs = t5_model.generate(**input_encodings, max_length=128)
        decoded_outputs = t5_tokenizer.batch_decode(outputs, skip_special_tokens=True)
        print("Output mẫu:", decoded_outputs)
        break

In [27]:
main()

Kích thước vocabulary: 32100


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Đã cập nhật embedding của T5 với custom embeddings qua projection layer.


I0000 00:00:1740435652.983292   26367 service.cc:148] XLA service 0x7f6f04002100 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740435652.983386   26367 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2025-02-25 05:20:52.997783: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1740435653.016223   26367 cuda_dnn.cc:529] Loaded cuDNN version 90501
I0000 00:00:1740435653.069580   26367 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Output mẫu: ['jump jump jump jump jump jump jump primit studies studies studies jump jump jump jump primit studies jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump jump studies studies studies studies studies studies studies studies studies studies studies studies studies studies', 'jump jump jump jump jump jump jump primit studies monastery studies monastery studies monastery studies monastery studies monastery studies monastery monastery primit monastery primit monastery primit monasteryected primit monasteryectedected jump jump jump jump jump jump jump jump jump ju

In [28]:
# class NLPModel:
#     def __init__(self, datanum: int = 1000) -> None:
#         # Load data
#         self.qfragments = json5.load(open(paths["qfragments"]))
#         self.questions = self._dataset(datanum)
        
#         self._prepare_datasets()

#     def dataset(datanum: int = 1000) -> pd.DataFrame:
#         if os.path.exists(paths["qtrain"]):
#             old_data = pd.read_csv(paths["qtrain"])
#             new_questions = pd.concat([old_data, generate_text(datanum)])
#         else:
#             new_questions = generate_text(datanum)
        
#         new_questions.to_csv(paths["qtrain"], index=False)
#         return new_questions